<h2>FINAL DATA PREPROCESSING & MODEL TRAINING</h2>

Script ini digunakan untuk mempreprocess data menjadi data final yang dapat digunakan untuk training model, dan training model LSTM itu sendiri. Secara umum, script ini meliputi:
<h4>Data Preprocessing</h4>
- Padding & Truncating data, pada dasarnya memastikan seluruh trj_id memiliki jumlah data yang sama, yakni 20 data, dengan menambahkan data yang kurang dengan 0 dan memotong data yang kelebihan dan mengambil 20 data paling belakang.
- Membagi data menjadi X (koordinat, keceptan, dll yang dimasukkan ke model untuk melatih model) dan y (koordinat yang benar untuk dibandingkan dengan koordinat hasil prediksi model). y akan diambil dari koordinat terakhir tiap trajectory.
- Membagi data menjadi training data dan testing data, dengan pembagian 90%/10%.
<br/>
<h4>Model Training</h4>
- Mendefinisikan dan melatih model LSTM berdasarkan data yang telah siap.
- Arsitektur, epoch, loss, optimizer, dan dll dapat dengan bebas diubah-ubah untuk mencoba model lain.
<br/>
Jika ingin melihat performa model pertama yang telah saya latih, maka model itu sudah disave dalam folder models dengan nama model_v1.keras. Model dapat ditest menggunakan script Model Test.ipynb

In [1]:
## Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import datetime

In [2]:
# Load Tensorboard
%load_ext tensorboard

In [35]:
## Read data from csv
resampled_data = pd.read_csv('../clean_data.csv', index_col='trj_id').drop('Unnamed: 1', axis=1)
print(resampled_data)

          rawlat      rawlng      speed     bearing  hour_of_day  day_of_week
trj_id                                                                       
1      -6.198042  106.769008   4.322800  179.920000           14            3
1      -6.200972  106.769202   8.014167  173.233333           14            3
1      -6.205394  106.769768  10.116136  171.477273           14            3
1      -6.210496  106.771217   9.307667  156.683333           14            3
1      -6.214969  106.773830  10.103333  139.777778           14            3
...          ...         ...        ...         ...          ...          ...
9999   -6.187751  106.845707  10.584667  329.600000            4            6
9999   -6.184123  106.843546   4.508780  324.512195            4            6
9999   -6.182706  106.842869   2.776724  287.137931            4            6
9999   -6.180504  106.842337   5.244333  326.850000            4            6
9999   -6.179029  106.841998   2.330952  231.095238            4

In [36]:
# Pad and truncate the timestamps in the dataframe
# Ini buat kita samain input modelnya, gw potong timestampnya jadi pasti ada 20 timestamp per sample. Kalo lebih dipotong, kalo kurang ditambahin 0 di depannya
# Pad value None --> pake koordinat pertama

def pad_truncate_dataframe(df, max_len, padding='pre', truncating='post', pad_value=None):
  # Split the dataframe by samples (first level of multi-index)
  samples = df.groupby(level=0)

  # Define a function to pad/truncate a single sample
  def pad_truncate_sample(sample):
    # Extract the values from a single sample
    values = sample.values
    first_element = values[0]
    first_element[2] = 0
    first_element[3] = 0

    # Pad/truncate automatically using pad_sequences
    if pad_value == None:
      padded_truncated = tf.keras.preprocessing.sequence.pad_sequences(
          [values], maxlen=max_len, padding=padding, truncating=truncating, value=first_element, dtype='float64'
      )[0]
    else:
      padded_truncated = tf.keras.preprocessing.sequence.pad_sequences(
          [values], maxlen=max_len, padding=padding, truncating=truncating, value=pad_value, dtype='float64'
      )[0]

    # Convert back to pandas dataframe
    return pd.DataFrame(padded_truncated, columns=sample.columns)

  # Apply the function to each sample and recreate the multi-index dataframe
  padded_df = samples.apply(pad_truncate_sample)
  return padded_df

df_new = pad_truncate_dataframe(resampled_data, 61)

In [37]:
## Convert the multi index dataframe to a numpy 3D array for better integration to TensorFlow (samples, timesteps, features)
numpy_data = df_new.to_xarray().to_array().to_numpy()
numpy_data = np.transpose(numpy_data, (1, 2, 0))
numpy_data.shape # Should be (55994, 21, 6) if 55994 samples, 21 timesteps, and 6 features

(55994, 61, 6)

In [38]:
## Split the data to x (feature values) and y (target values)
simple_data = numpy_data[:, :, 0:2]
x_data = simple_data[:, :-1, :]
y_data = simple_data[:, -1, :]

In [39]:
# Split the data to train and test splits (train to train the model, test to test the model on new data after trained)
test_size = 0.1

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_size)

In [40]:
## Define normalization layer
normalize_layer = tf.keras.layers.Normalization(axis=-1, input_shape=(60, 2))
normalize_layer.adapt(simple_data)

c:\Users\adrie\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\preprocessing\normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
## Get Mean and Variance of Normalization layer to use for normalizing y and denormalizing later
normalize_weights = normalize_layer.get_weights()

mean_variance = np.array([normalize_weights[0][0:2], normalize_weights[1][0:2]])

np.save('../mean_variance_C.npy', mean_variance)

In [42]:
## Normalize y
normalize_y = tf.keras.layers.Normalization(mean=mean_variance[0], variance=mean_variance[1])

y_train = normalize_y(y_train)
y_test = normalize_y(y_test)
print(y_train)

tf.Tensor(
[[ 0.9400123  -0.88482916]
 [ 0.07942872  1.3187108 ]
 [-2.0560253   0.8154418 ]
 ...
 [ 1.0032728   0.7083409 ]
 [ 0.17912859  0.27326536]
 [-0.9182232  -0.5357855 ]], shape=(50394, 2), dtype=float32)


In [43]:
## Save test data for model testing
np.save('../x_test_C.npy', x_test)
np.save('../y_test_C.npy', y_test)

In [44]:
# Clear any previous models
tf.keras.backend.clear_session()

# Define the model
model = tf.keras.Sequential()
model.add(normalize_layer)
model.add(tf.keras.layers.LSTM(32, return_sequences=True))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(2))


# Define optimizer
lr = 1e-3
opt = tf.keras.optimizers.Adam(learning_rate=lr)

model.compile(loss='huber', optimizer=opt, metrics=['mae', 'mse'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 60, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 60, 32)         │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,871 (50.28 KB)

 Trainable params: 12,866 (50.26 KB)

 Non-trainable params: 5 (24.00 B)

In [45]:
## Define LR scheduling (optional if want to use or not)
start_lr = lr
min_lr = 0.00001
max_lr = 0.001
rampup_epochs = 0
sustain_epochs = 0
exp_decay = 0.0

# Define the scheduling function
def schedule(epoch):
  def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
    if epoch < rampup_epochs:
      lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
      lr = max_lr
    else:
      lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    return lr
  return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)

In [46]:
## Define callbacks and fit the model
log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(schedule)

model.fit(
  x=x_train, 
  y=y_train,
  epochs=200,
  validation_data=(x_test, y_test),
  callbacks=[tensorboard_callback, early_stopping,]) #Can add lr_scheduler

model.evaluate(x_test, y_test)

Epoch 1/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 51s 31ms/step - loss: 0.0243 - mae: 0.1126 - mse: 0.0608 - val_loss: 8.7136e-04 - val_mae: 0.0302 - val_mse: 0.0017
Epoch 2/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - loss: 7.2641e-04 - mae: 0.0275 - mse: 0.0015 - val_loss: 4.2815e-04 - val_mae: 0.0219 - val_mse: 8.5630e-04
Epoch 3/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 51s 32ms/step - loss: 3.4779e-04 - mae: 0.0190 - mse: 6.9559e-04 - val_loss: 1.8521e-04 - val_mae: 0.0131 - val_mse: 3.7042e-04
Epoch 4/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - loss: 2.0228e-04 - mae: 0.0146 - mse: 4.0457e-04 - val_loss: 7.6650e-04 - val_mae: 0.0204 - val_mse: 0.0015
Epoch 5/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 48s 30ms/step - loss: 2.5942e-04 - mae: 0.0148 - mse: 5.1884e-04 - val_loss: 9.6759e-05 - val_mae: 0.0099 - val_mse: 1.9352e-04
Epoch 6/200
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 51s 32ms/step - loss: 1.3586e-04 - mae: 0.0117 - mse: 2.7171e-04 - val_loss: 9.6745e-05 - val_mae: 0.0101 - val_mse: 1.934

[0.00019356864504516125, 0.013347640633583069, 0.0003871372900903225]

In [48]:
## Save the model
model.save('../models/model_v0.keras')